In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import scipy.stats

from IPython.display import display
import ipywidgets as widgets
from functools import partial
slider_layout = widgets.Layout(width='600px', height='20px')
slider_style = {'description_width': 'initial'}
IntSlider_nice = partial(widgets.IntSlider, style=slider_style, layout=slider_layout, continuous_update=False)
FloatSlider_nice = partial(widgets.FloatSlider, style=slider_style, layout=slider_layout, continuous_update=False)
SelSlider_nice = partial(widgets.SelectionSlider, style=slider_style, layout=slider_layout, continuous_update=False)

# Mundo de datos

Los avances tecnológicos recientes nos permiten **medir, almacenar y enviar** datos de toda índole

- Operaciones industriales
- Comercio
- Entretenimiento 
- Datos públicos y gubernamentales
- Datos médicos
- Ciencia: Genómica, Astronomía, Simulaciones, etc
- Vehículos autónomos
- Internet de las cosas

> Los datos crudos tienen poco valor, necesitamos extraer información a partir de los datos

- ¿Cómo se comportan mis datos? ¿Cúales son las observaciones más cómunes? ¿Qué datos son más relevantes?
- ¿Cúal es la diferencia entre dos muestras? ¿Son las diferencias que observo reales o productos del ruido?
- ¿Qué tan probable es que ocurra el suceso $X$? 
- ¿Qué tan arriesgado es tomar la decisión $Y$?

> La **estadística** nos da herramientas para entender los procesos y tomar decisiones


En esta clase aprenderemos a usar [`scipy.stats`](https://docs.scipy.org/doc/scipy/reference/stats.html) y [`numpy.random`](https://docs.scipy.org/doc/numpy/reference/routines.random.html) para resolver problemas de **inferencia estadística**

Pero antes, algunos fundamentos:

## Fundamentos: Incerteza

Fuentes de incerteza:
1. Estocasticidad inherente: Sistemas con [dinámicas aleatorias](https://en.wikipedia.org/wiki/Uncertainty_principle)
1. Observación incompleta: Sistema determinista que parece estocástico [cuando no se observa completamente](https://en.wikipedia.org/wiki/Monty_Hall_problem)
1. Modelamiento incompleto: Los [supuestos y aproximaciones del sistema](https://en.wikipedia.org/wiki/Discretization) introducen incerteza 

¿Cómo representamos la incerteza?

> Teoría de probabilidades

Nos proporciona reglas formales para determinar la verosimilitud de una proposición versus otras

## Fundamentos: Variables aleatorias 

- **V.A.** Es una variable que, al ser observada, puede tomar diferentes valores

    - La denotamos como $X$
    
    - Sus observaciones (realizaciones) son $x\sim X$
    
    - Sus realizaciones tienen un dominio $x \in \mathcal{X}$
    
    - La probabilidad de observar $x$ es $P(X=x)$
    
    - Puede ser discreta o continua
    

    
- El comportamiento de $X$ está dictado por 
    - Función de masa de probabilidad (para $X$ discreta)
    
    $P(X=x) \in [0, 1]$
    
    $\sum_{x\in\mathcal{X}} P(X=x) = 1$    
    
    - Función de densidad de probabilidad (para $X$ continua)
    
    $f(x) \geq 0$
    
    $\int_{x\in\mathcal{X}} f(x) \,dx = 1$, 
    
    $P(a\leq X \leq b) = F(b) - F(a) = \int_{a}^{b} f(x) \,dx$
    
        - Función de densidad acumulada: 
        
        $F(a)  = \int_{-\infty}^{a} f(x) \,dx$
        
        
> **Importante:** Sólo en el caso discreto la distribución puede interpretarse como probabilidad

In [ ]:
plt.close('all'); fig, ax = plt.subplots(1, 2, figsize=(8, 3))
dt=1e-4; x = np.arange(-5, 5, step=dt)

from scipy.special import erf
gaussian_pdf = lambda x, mu=0, s=1: np.exp(-0.5*(x-mu)**2/s**2)/(s*np.sqrt(2*np.pi))
gaussian_cdf = lambda x, mu=0, s=1: 0.5 + 0.5*erf((x-mu)/(s*np.sqrt(2)))

def update_plot(x_r):
    xi, xf = x_r
    for axis in ax:
        axis.cla(); 
        axis.set_xlim([-5, 5]);
    ax[0].plot(x, gaussian_pdf(x)); ax[1].plot(x, gaussian_cdf(x));
    xrange = np.arange(xi, xf, step=dt)
    ax[0].fill_between(xrange, 0, gaussian_pdf(xrange), alpha=0.5)
    ax[1].scatter([xi, xf], [gaussian_cdf(xi), gaussian_cdf(xf)], s=100, c='k', zorder=100)
    ax[1].text(xi+0.5, gaussian_cdf(xi), "Init"); ax[1].text(xf+0.5, gaussian_cdf(xf), "End")
    ax[0].set_title("$\int_{x_i}^{x_f} f(x) dx$ = %0.4f" %(np.sum(gaussian_pdf(xrange))*dt))
    area = gaussian_cdf(xf) - gaussian_cdf(xi)
    ax[1].set_title("$F(x_f) - F(x_i)$ = %0.4f" %(area if area >= 0 else 0))

widgets.interact(update_plot, 
         x_r=widgets.FloatRangeSlider(description="$x_i, x_f$", min=-5, max=5, value=[0,0]));

## Probabilidad conjunta, marginal y condicional

La probabilidad de dos eventos $X$ e $Y$ se caracteriza con la distribución conjunta $P(X, Y)$

A partir de la conjunta se pueden obtener las probabilidades marginales
$$
P(X=x) = \sum_{y \in \mathcal{Y}} P(X=x, Y=y) 
$$

Usando la conjunta y las marginales podemos obtener las probabilidades condicionales
$$
P(Y=y|X=x) = \frac{P(X=x, Y=y)}{P(X=x)}
$$

## Regla de la cadena

$$
P(x_1, x_2, x_3) = P(x_1) P(x_2|x_1) P(x_3|x_2, x_1)
$$

## Ley de probabilidad total

Si el espacio de probabilidad está particionado en $N$ pedazos entonces
$$
P(X=x) = \sum_{i=1}^N P(X=x|A=a_i) P(A=a_i)
$$

## Teorema de Bayes

$$
P(y | x) = \frac{P(x|y) P(y)}{P(x)} = \frac{P(x|y) P(y)}{\sum_{y\in\mathcal{Y}} P(x|y) P(y)}
$$

## Independencia

Dos V.A. son independientes si

$$
P(x, y)  = P(x)P(y)
$$

> Saber que ocurrió $x$ no me sirve da nada para saber si ocurrió $y$

Dos V.A. son condicionalmente independientes si

$$
P(x, y|z)  = P(x|z)P(y|z)
$$

## Descriptores de las distribuciones

Podemos describir una variable aleatoria $X$ con $x \sim f(x)$ usando

### Valor esperado

El valor medio de $X$

$$
\mathbb{E}_{x\sim f(x)} [ X ] = \int_{x\in \mathcal{X}} x f(x)  \,dx
$$

### Varianza 

La dispersión de $X$ en torno a su valor medio

$$
\text{Var}[X]  = \mathbb{E}_{x\sim f(x)} \left[\left(X - \mathbb{E}[X] \right)^2 \right]
$$

la relación lineal entre $X$ e $Y$

$$
\text{Cov}[X, Y]  = \mathbb{E}_{x\sim f_X(x), y\sim f_Y(y)} \left[\left(X - \mathbb{E}[X] \right) \left(Y - \mathbb{E}[Y] \right)^T \right]
$$


### Momentos estadísticos

$$
m_k [X] = \mathbb{E}_{x\sim f(x)} [ X^k ]
$$

- Tercer momento: Simetría 
- Cuarto momento: Cúrtosis


In [ ]:
fig, ax = plt.subplots(2, figsize=(7, 3), tight_layout=True)
x = np.linspace(-4, 4, num=1000)

def update_plot(loc, scale, skew, shape):
    distribution1 = scipy.stats.skewnorm(skew, loc=loc, scale=scale)
    distribution2 = scipy.stats.gennorm(shape, loc=loc, scale=scale*np.sqrt(2))
    [ax_.cla() for ax_ in ax]
    ax[0].plot(x, distribution1.pdf(x))
    ax[1].plot(x, distribution2.pdf(x))
    
widgets.interact(update_plot, loc=FloatSlider_nice(min=-3, max=3), 
                 scale=FloatSlider_nice(min=0.01, max=2., value=1), 
                 skew=FloatSlider_nice(min=-5, max=5, value=0),
                 shape=FloatSlider_nice(min=0.1, max=10, value=2));

## Algunas distribuciones de probabilidad

| Distribución | Fenomeno que representa | Ejemplo |
| --- | --- | --- |
| **Bernoulli** | Evento binario  | Lanzamiento de una moneda |
| **Binomial** | Multiples eventos binarios independientes | |
| **Categórica** | Evento con $k$ valores posibles | Lanzamiento de un dado, Ruleta |
| **Poisson** | Conteo de eventos ocurridos en un período de tiempo | Cantidad de alumnos que llegan entre 9:50 y 10:00| 
| **Exponencial** | Valor continuo positivo | Tiempo de espera entre eventos|
| **Gamma** | Valor continuo positivo | Tiempos de espera hasta que ocurren $n$ eventos|
| **Beta** | Valor continuo en $[0, 1]$ |  Tiempo para completar una tarea, proporciones|
| **Normal/Gaussiana** | Valor continuo ubicado en la vecindad de un valor central| [Demasiados](https://galtonboard.com/probabilityexamplesinlife)|
| **Uniforme** | Valor discreto/continuo acotado a un rango, todos con igual probabilidad de ocurrencia| |



<img src="https://thumbs.gfycat.com/AggressiveAromaticBuckeyebutterfly-size_restricted.gif">

## En detalle: Multinomial




## En detalle: Gaussiana/Normal Multivariada

- Dominio $x\in \mathbb{R}^d$
- Parámetros: 
    - Media $\mathbb{E}[X] = \mu \in \mathbb{R}^d$ 
    - Covarianza $\mathbb{E}[(X-\mu)(X-\mu)^T] = \Sigma \in \mathbb{R}^{d\times d}$ (semidefinida positiva)
- Función de densidad de probabilidad
$$
p(x| \mu, \Sigma) = \frac{1}{ \sqrt{(2 \pi)^d} |\Sigma |} \exp \left ( -\frac{1}{2} (x-\mu)^T \Sigma^{-1} (x-\mu) \right)
$$

- Casos especiales
    - Covarianza diagonal: $\Sigma = [\sigma_1^2, \sigma_2^2, \ldots, \sigma_d^2] I$
    - Covarianza isotrópica (esférica): $\Sigma = \sigma^2 I$
    - Normal estándar: $\mu = [0, 0, \ldots, 0]$, $\Sigma = I$

In [ ]:
fig, ax = plt.subplots(1, figsize=(5, 4))

def update_plot(mu1, mu2, s1, s2, rho, seed):
    ax.cla()
    np.random.seed(seed)
    mu = np.array([mu1, mu2]); s = np.diag(np.array([s1, s2]))
    rot_mat = [[np.cos(rho), -np.sin(rho)], [np.sin(rho), np.cos(rho)]]
    L = np.dot(rot_mat, s)
    x = np.random.multivariate_normal(mean=mu, cov=np.dot(L, L.T), size=5000)
    ax.scatter(x[:, 0], x[:, 1], s=5, alpha=0.5)
    ax.set_xlim([-3, 3]); ax.set_ylim([-3, 3])
    ax.set_aspect('equal')
    
widgets.interact(update_plot, mu1=FloatSlider_nice(min=-2, max=2), mu2=FloatSlider_nice(min=-2, max=2),
                 s1=FloatSlider_nice(value=1, min=0.1, max=2), s2=FloatSlider_nice(value=1, min=0.1, max=2),
                 rho=FloatSlider_nice(value=0, min=-np.pi/2, max=np.pi/2), seed=IntSlider_nice());

## Ley de los grandes números

Si $X_1, X_2, \ldots, X_N$ son V.A independientes e idénticamente distribuidas (iid) con media $\mu$  entonces

$$
\bar X = \frac{1}{N} (X_1 + X_2 + \ldots + X_N)
$$

Entonces $\bar X \to \mu$ cuando $N \to \infty$

## Teorema central del límite

Si $X_1, X_2, \ldots, X_N$ son V.A independientes e idénticamente distribuidas (iid) 

$$
\bar X \sim \mathcal{N}(\mu, \sigma^2/N)
$$

> Sin importar su distribución original, si sumo muchas VA iid entonces la suma se distribuye normal

# Estadística

> Disciplina científica que engloba métodos que utilizan datos para responder preguntas

> Entender fenómenos complejos a partir de observaciones parciales 

> Inferir la probabilidad de un evento basándonos en una muestra de datos

> Disciplina científica dedicada al desarrollo y estudio de métodos para recopilar, analizar e interpretar a partir de datos



## Estadística descriptiva

Sea una muestra de datos. Para entenderla mejor podemos empezar describiendola:
- ¿Discreto o continuo? ¿No-negativo?
- ¿Dónde está localizada? **Media**
- ¿Cuál es su disperción? **Varianza**
- ¿Son las colas iguales o distintas? **Simetría** (*Skewness*)
- ¿Son las colas largas o cortas? **Curtosis** (*Kurtosis*)
- ¿Tiene una moda o múltiples modas?
- ¿Cuantiles? ¿Percentiles? 
- ¿Existen *outliers*?

Podemos responder estas preguntas usando [estadísticos de resumen](https://docs.scipy.org/doc/scipy/reference/stats.html#summary-statistics)

In [ ]:
x = np.linspace(-11, 10, num=1000)
px = 0.7*gaussian_pdf(x, mu=-4, s=2) + 0.3*gaussian_pdf(x, mu=3, s=2)
N = 1000; 
np.random.seed(0)
hatx = np.concatenate((-4 + 2*np.random.randn(int(0.7*N)), 
                       (3 + 2*np.random.randn(int(0.3*N)))))

scipy.stats.describe(hatx)

En ciertos casos podemos responder estas preguntas graficamente usando:

### Histograma

- Es una representación numérica de una distribución
- Nos permite visualizar las características de la distribución
- Se construye dividiendo el dominio en **bines** y contando los datos que caen en cada **bin**
- Está definido por la posición y tamaño de los bines

In [ ]:
plt.close('all'); fig, ax = plt.subplots(figsize=(7, 4))

def update_plot(nbins): 
    ax.cla()
    ax.plot(x, px, 'k-', linewidth=4, alpha=0.8)
    hist, bin_edges = np.histogram(hatx, bins=nbins, density=True)
    ax.bar(bin_edges[:-1], hist, width=bin_edges[1:] - bin_edges[:-1], 
           edgecolor='k', align='edge', alpha=0.8)
    ax.set_xlabel('x')
    
widgets.interact(update_plot, nbins=SelSlider_nice(options=[2, 5, 10, 15, 20, 50], value=5));

### Kernel density estimation (KDE)

Mismo objetivo que el histograma pero
1. Cada dato es "su propio bin"
1. Los bines se pueden traslapar
1. No se escoge la posición o fronteras de bines, solo su ancho

Para un set de observaciones unidimensionales $\{x_i\}_{i=1,\ldots, N}$ con dominio continuo

$$
\hat f_h(x) = \frac{1}{Nh} \sum_{i=1}^N \kappa \left ( \frac{x - x_i}{h} \right)
$$

donde 
- $h$ es el **ancho de banda del kernel** o **tamaño de kernel**
- $\kappa(u)$ es una **función de kernel** que debe ser positiva, con media cero e integrar a la unidad

Por ejemplo  el famoso kernel Gaussiano

$$
\kappa(u) = \frac{1}{\sqrt{2\pi}} \exp \left ( - \frac{u^2}{2} \right),
$$

Ojo: 
- Usar un kernel gaussiano no es lo mismo que asumir que los datos se distribuyen gaussianos
- Se puede usar un kernel gaussiano en datos que no se distribuyen gaussianos

In [ ]:
from sklearn.neighbors.kde import KernelDensity
plt.close('all'); fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(x, px, 'k-', linewidth=4, alpha=0.8)
ax.scatter(hatx, np.zeros_like(hatx), marker='+', c='k', s=20, alpha=0.1)
ax.set_xlabel('x')
line_kde = ax.plot(x, np.zeros_like(x))
#hs = 0.9*np.std(hatx)*N**(-1/5)
def update(k): 
    kde = scipy.stats.gaussian_kde(hatx, bw_method=lambda kde: k*kde.silverman_factor() )
    line_kde[0].set_ydata(kde.pdf(x))
    
widgets.interact(update, k=SelSlider_nice(description="$k=h/h_s$", options=[1/8, 1/4, 1/2, 1, 2, 4], value=1));

Más opciones de kernels en [`sklearn.neighbors.KernelDensity`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html#sklearn.neighbors.KernelDensity)

# Inferencia estadística

> Extraer conclusiones a partir de hechos a través de una premisa científica

- Hechos: Datos
- Premisa: Modelo probabilístico
- Conclusión: Una cantidad no observada que es interesante

> Cuantificar la incerteza de la conclusión dado los datos y el modelo 

## Métodos paramétricos